In [1]:
# imports
import numpy as np
import os
import os.path
import pandas as pd
pd.options.display.max_colwidth = 100
import cv2
import tensorflow as tf
import pickle
import copy
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [58]:
predictions_per_tasks_directory_path = '/home/acarlier/code/OrnithoScope/saved_models/tasks'
input_csv_path = '/home/acarlier/code/OrnithoScope/data/csv/input.csv'
tasks_csv_path = '/home/acarlier/code/OrnithoScope/data/csv/Ornithotasks - CVAT_task.csv'
dataset_test_groupedby_path = '/home/acarlier/code/OrnithoScope/data/csv/dataset_test_groupedby.csv'
columns_names = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"]
bird_classes = pickle.load( open( "/home/acarlier/code/OrnithoScope/data/pickles/basic_data/bird_classes.p", "rb" ) )
df_groupedby = pd.read_pickle("/home/acarlier/code/OrnithoScope/data/pickles/basic_data/df_dataset_test_groupedby.p")

In [59]:
def load_pred_per_tasks(predictions_per_tasks_directory_path, tasks_csv_path):
    '''
    Generate a list of dictionary : 
    [
        {
            'task_name': task_20210705-07_balacet,
            'site': 'Francon',
            'day': ['01-03-2021'],
            'pred': pickle.load(f"{path_tasks_directory}/{task_name}.p")
        }
    ]
    '''
    list_of_pred_per_tasks = []
    df_tasks = pd.read_csv(tasks_csv_path)
    for file in os.listdir(predictions_per_tasks_directory_path):
        result = {}
        # Generate task_name
        task_name = '_'.join(file.split('.')[0].split('_')[2:])
        result['task_name'] = task_name
        # Generate site
        site_name = df_tasks[df_tasks['task_name'] == task_name]['Site'].values[0]
        result['site'] = site_name
        # Generate day
        day_name = df_tasks[df_tasks['task_name'] == task_name]['Day'].values[0]
        result['day'] = day_name
        # Adding pred
        pred = pickle.load( open( f"{predictions_per_tasks_directory_path}/{file}", "rb" ) )
        result['pred'] = pred
        list_of_pred_per_tasks.append(result)
    return list_of_pred_per_tasks

In [135]:
def generate_list_of_pred_per_tasks(predictions_per_tasks_directory_path):
    list_pred_per_task = []
    for file in os.listdir(predictions_per_tasks_directory_path):
        pred = pickle.load( open( f"{predictions_per_tasks_directory_path}/{file}", "rb" ) )
        list_pred_per_task.append(pred)
    return list_pred_per_task
list_pred_per_task = generate_list_of_pred_per_tasks(predictions_per_tasks_directory_path)
list_pred_per_task[0]

[[],
 [{'bounding_box': array([0.34909683, 0.59888756, 0.7009031 , 0.832687  ], dtype=float32),
   'class_id': 0.0,
   'score': 0.80078125}],
 [{'bounding_box': array([0.3616184 , 0.18894264, 0.6049975 , 0.4349998 ], dtype=float32),
   'class_id': 0.0,
   'score': 0.5859375}],
 [{'bounding_box': array([0.33405524, 0.48947933, 0.54489934, 0.6481116 ], dtype=float32),
   'class_id': 12.0,
   'score': 0.33203125}],
 [],
 [{'bounding_box': array([0.40891162, 0.01906559, 0.9261867 , 0.3628304 ], dtype=float32),
   'class_id': 0.0,
   'score': 0.5},
  {'bounding_box': array([0.34090033, 0.5094978 , 0.544198  , 0.6522017 ], dtype=float32),
   'class_id': 5.0,
   'score': 0.4765625}],
 [{'bounding_box': array([0.3452539, 0.5072502, 0.5398444, 0.6515244], dtype=float32),
   'class_id': 5.0,
   'score': 0.4140625},
  {'bounding_box': array([0.36796358, 0.1236061 , 0.88523865, 0.38224363], dtype=float32),
   'class_id': 0.0,
   'score': 0.4140625}],
 [],
 [{'bounding_box': array([0.40547967, 0.64

In [60]:
#list_of_pred_per_tasks = load_pred_per_tasks(predictions_per_tasks_directory_path,tasks_csv_path)

In [178]:
def convert_id_to_label_name(list_of_results,list_of_classes):
    '''
    list_of_results as: 
    [[{'bounding_box': array([0.08540744, 0.25474447, 0.68958795, 0.4716437 ], dtype=float32),
    'class_id': 7.0,
    'score': 0.6484375}],
    '''
    results = copy.deepcopy(list_of_results)
    for i in range(len(results)):
        for result in results[i]:
            id = int(result['class_id'])
            result['class_id'] = list_of_classes[id]
    return results
def get_list_of_label_from_prediction(list_of_results):
    results = copy.deepcopy(list_of_results)
    l_final = []
    for i in range(len(results)):
        l = []
        for result in results[i]:
            label = result['class_id']
            l.append(label)
        l_final.append(l)
    return l_final
def clean_list_of_predict_per_tasks_for_plotting(list_of_pred_per_tasks,list_of_classes):
    list_of_pred_per_tasks = copy.deepcopy(list_of_pred_per_tasks)
    for i in range(len(list_of_pred_per_tasks)):
        list_of_pred_per_tasks[i] = convert_id_to_label_name(list_of_pred_per_tasks[i], list_of_classes)
    return list_of_pred_per_tasks

In [179]:
list_pred_per_tasks_cleaned = clean_list_of_predict_per_tasks_for_plotting(list_pred_per_task, bird_classes)

In [184]:
list_pred_per_tasks_cleaned[0][1]

[{'bounding_box': array([0.34909683, 0.59888756, 0.7009031 , 0.832687  ], dtype=float32),
  'class_id': 'MESCHA',
  'score': 0.80078125}]

# Calculate metrics per tasks

In [181]:
def get_list_of_label_from_prediction(list_of_results):
  results = copy.deepcopy(list_of_results)
  l_final = []
  for i in range(len(results)):
      l = []
      for result in results[i]:
          label = result['class_id']
          l.append(label)
      l_final.append(l)
  return l_final
def evaluate_multiple_detections_try_version(true_labels, pred_labels):
  TP = []
  FP = copy.deepcopy(pred_labels)
  FN = copy.deepcopy(true_labels)
  print('in method - pred ',pred_labels, 'pred ',true_labels)
  for pl in pred_labels:
    if pl in true_labels:
      try:
        FP.remove(pl)
      except ValueError:
        "not in the list"
      try:
        FN.remove(pl)
      except ValueError:
        "not in the list"
      try:
        TP.append(pl)
      except ValueError:
        "not in the list"
    print('in method - Results \n', 'TP ',TP,'\n FN',FN,'\n FP',FP)
  return TP, FN, FP
def get_precision_recall_from_prediction(list_of_results, df_input_test, list_of_classes):
    class_metrics = []
    for classes in list_of_classes:
        class_metrics.append({'TP':0, 'FP':0, 'FN':0})
    for i in range(len(list_of_results)):
        pred_labels = list_of_results[i]
        print('main - pred', pred_labels)
        true_labels = df_input_test.iloc[i]['label']
        print('main - true', true_labels)
        TP, FN, FP = evaluate_multiple_detections_try_version(true_labels, pred_labels)
        #print(TP, FN, FP)
        for lab in TP:
            class_metrics[list_of_classes.index(lab)]['TP'] += 1
        for lab in FN:
            class_metrics[list_of_classes.index(lab)]['FN'] += 1
        for lab in FP:
            class_metrics[list_of_classes.index(lab)]['FP'] += 1
        print('Class metrics iteration ', class_metrics)
    return class_metrics

In [182]:
def get_list_of_pred(list_of_pred_per_tasks_cleaned):
    list_of_pred_per_tasks_cleaned = copy.deepcopy(list_of_pred_per_tasks_cleaned)
    for task in list_of_pred_per_tasks_cleaned: 
        list_of_label_from_prediction = get_list_of_label_from_prediction(task['pred'])
        class_metrics = get_precision_recall_from_prediction(list_of_label_from_prediction, df_groupedby, bird_classes)
        task['class_metrics'] = class_metrics
    return list_of_pred_per_tasks_cleaned

In [183]:
list_of_pred_per_tasks_cleaned_class_metrics = get_list_of_pred(list_pred_per_tasks_cleaned)
list_of_pred_per_tasks_cleaned_class_metrics[0]['class_metrics']

TypeError: list indices must be integers or slices, not str

In [81]:
results_predictions = pickle.load( open( "/home/acarlier/code/OrnithoScope/saved_models/results_EfficientDet1.p", "rb" ) )
results_predictions

[[{'bounding_box': array([0.08540744, 0.25474447, 0.68958795, 0.4716437 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.6484375}],
 [{'bounding_box': array([0.4092438, 0.5277908, 1.0296512, 0.7281122], dtype=float32),
   'class_id': 0.0,
   'score': 0.375}],
 [{'bounding_box': array([0.5509056, 0.5129066, 0.9888495, 0.9870934], dtype=float32),
   'class_id': 0.0,
   'score': 0.9296875}],
 [{'bounding_box': array([0.39843464, 0.30333823, 0.6451436 , 0.5669474 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.84765625}],
 [],
 [{'bounding_box': array([0.41062987, 0.13668333, 0.8865938 , 0.38762027], dtype=float32),
   'class_id': 0.0,
   'score': 0.91796875}],
 [{'bounding_box': array([0.46652228, 0.13806312, 0.668348  , 0.5108628 ], dtype=float32),
   'class_id': 7.0,
   'score': 0.8125},
  {'bounding_box': array([0.8450288, 0.8830899, 0.993529 , 0.998536 ], dtype=float32),
   'class_id': 0.0,
   'score': 0.76953125}],
 [{'bounding_box': array([0.43046698, 0.2513759 , 0.6197371 